In [2]:
%%capture output 
%pip install boto3 pandas pyarrow python-dotenv h3 geopandas matplotlib shapely

In [3]:
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
import io
import h3
import geopandas as gpd
from shapely import points


In [4]:
load_dotenv('../wb_aws.env')

True

In [5]:
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')
)

s3_client = session.client('s3')

In [22]:
bucket_name = 'wbg-geography01'
folder_path = 'Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/'

In [23]:
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
for obj in response['Contents']:
    print(f"{obj['Key']} {obj['Size']} Bytes")


Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/01/DNB_npp_20120119-20120131.parquet 307164499 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/02/DNB_npp_20120201-20120229.parquet 309784561 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/03/DNB_npp_20120301-20120331.parquet 309473033 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/04/DNB_npp_20120401-20120430.parquet 307531491 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/05/DNB_npp_20120501-20120531.parquet 299583037 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/06/DNB_npp_20120601-20120630.parquet 294118936 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/07/DNB_npp_20120701-20120731.parquet 300869059 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/08/DNB_npp_20120801-20120831.parquet 306292863 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/09/DNB_npp_20120901-20120930.parquet 309040259 Bytes
Space2Stats/parquet/GLOBAL/NTL_VIIRS_LEN/2012/10/DNB_npp_20121001-20121031.parquet 308995231 Bytes
Space2Stat

In [27]:
obj = s3_client.get_object(Bucket=bucket_name, Key=folder_path+'2012/01/DNB_npp_20120119-20120131.parquet')
body = obj['Body'].read()

buffer = io.BytesIO(body)
df = pd.read_parquet(buffer)
df.head()

,SUM,MIN,MAX,MEAN,id
0,-1.000000,-1.00,-1.00,-1.000000,8600cd1b7ffffff
1,570.690063,0.96,1.31,1.070713,8601638a7ffffff
2,1090.969971,1.62,1.74,1.678415,860096a0fffffff
3,856.640015,1.12,29.42,1.874486,860128c07ffffff
4,515.559998,0.79,1.27,0.976439,860162a17ffffff


### Open Questions

- What does the VIIRS data represent? 
- Do we want to keep the mean, min, max?
- Do we see any issues with creating a separate column within our combined dataset for each month under the format `VIIRS_{year}_{month}_{aggregation_method}`?
- Can we add the details to the [datasets documentation](https://docs.google.com/spreadsheets/d/1bPWyZ9m-QwL3R4Ou-Alc0PEBG0S2za3ZeoJYEIyumI4/edit?gid=0#gid=0)? It wasn't initially included.
- Should we use another convention `NaN`, or `null` instead of `-1` for no data values when numerical values? What about other types of values?
- Who owns pivoting the data and combining it in the same file?